In [9]:
using Serialization, Turing
include("ode_problem.jl");
include("target_probability.jl");
include("bayesian_inference.jl");

### Define Target Distribution for Sampling

In [10]:
#first input we need are data points and each point's associated standard deviation
experiment_timecourse = deserialize("outputs/000_processed_active_G_timecourse.dict")
experiment_dose_response = deserialize("outputs/000_processed_active_G_dose_response.dict");
#reshape for input to target probability 
data_points = vcat(experiment_timecourse["response"], experiment_dose_response["response"])
std_dev = vcat(fill(experiment_timecourse["standard_error"], length(experiment_timecourse["response"])), 
fill(experiment_dose_response["standard_error"], length(experiment_dose_response["response"])))

#next input we need is an ode problem
odesys, u0, tspan, p = return_ode_problem_default_inputs()
odeprob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p);

#also need regularization hyperparameters
#note - must convert from M to molecules to match units
avogadros_constant = 6.022e23 
regularization_hyperparams = Dict("mean" => log10(1.01e-7*avogadros_constant), "std_dev" => 0.8, "lambda" => 1)
regularization_hyperparams_theoretical = Dict("mean" => log10(5.0e-9*avogadros_constant), "std_dev" => 1, "lambda" => 1)

#and ode solver inputs for both timecourse simulation and dose response simulation
solver_inputs_timecourse = return_ode_problem_solver_default_inputs("timecourse")
solver_inputs_dose_response = return_ode_problem_solver_default_inputs("dose_response")

#finally, ligand concentrations for dose respones simulation and normalization
ligand_doses = experiment_dose_response["ligand_stimulation (molecules)"];
normalization_dose = experiment_dose_response["normalize_to_response_at_dose"];

In [11]:
#target probability definition
target_distribution_regularized = logprob_regularized(data_points, odeprob, std_dev, regularization_hyperparams, solver_inputs_timecourse, 
solver_inputs_dose_response, ligand_doses, normalization_dose);

target_distribution_regularized_theoretical = logprob_regularized(data_points, odeprob, std_dev, regularization_hyperparams_theoretical, solver_inputs_timecourse, 
solver_inputs_dose_response, ligand_doses, normalization_dose);

target_distribution_unregularized = logprob_unregularized(data_points, odeprob, std_dev, solver_inputs_timecourse, 
solver_inputs_dose_response, ligand_doses, normalization_dose);

### Run Affine Invariant Sampler

In [12]:
#affine invariant sampler parameters
n_ensemble = 1
n_walkers = 100
n_iterations = 100

#target probability
target_probability = Dict("unregularized"=>target_distribution_unregularized, 
"regularized"=>target_distribution_regularized, "regularized_theoretical"=>target_distribution_regularized_theoretical)

#initial run for both regularized and unregularized 
for i in ["unregularized"]
    affine_invariant_mcmc_firstrun(n_ensemble, n_walkers, n_iterations, target_probability[i], i)
end

#generate 50,000 samples per walker for all target probabilities
n_subchains = 10
for i in ["unregularized", "regularized_theoretical","regularized"]
    affine_invariant_mcmc(n_ensemble, n_walkers, n_iterations, target_probability[i], n_subchains, i)
end

LoadError: SystemError: opening file "outputs/posterior_samples_unregularized_ensemble1_walkers100_iter100_9.jls": No such file or directory